# DNN 학습의 4가지 문제점과 테크닉
|   | Issue                                                                | Techniques |
|---|----------------------------------------------------------------------|------------|
| 1 | vanishing / exploding grad 로 인한 lower layer 학습의 어려움. | 1. Weigth Init<br>2. Nonsaturating activation ftns<br>3. Batchnorm<br>4. Gradient Clipping |
| 2 | not enough labeled train data (라벨링 작업이 너무 비싼 경우 포함) | 1. Transfer learning <br>2. Unsupervised pretraining <br>3. Pretraining on an auxiliary task |
| 3 | 느린 학습 속도 | Faster Optimizers |
| 4 | 학습데이터에 대한 오버피팅 (양이 적거나 노이즈가 많은 경우) | Regularization |

# ISSUE 1. Vanishing / Exploding Gradients
* logistic sigmoid activation ftn + normal dist weight init --> unstable grad

## 1. Weight init: Glorot and He Init
* logistic sigmoid activation ftn + normal dist weight init --> unstable grad
* [paper](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf) 에 따르면 다음 두가지가 만족될 때 grad 가 stable 하게 유지되어 학습이 잘 된다. 
    * var of out of each layer == var of input
    * var of grad == var of grad next time
* 페이퍼에서 제시한 weight init 방법은 *Xavier init* 또는 *Glorot init* 으로 다음과 같다. *(fan-in, fan-out of the layer: the numbers of input and output neurons)*
    * normal dis w/ mean 0 and var $\sigma^2 = \frac{1}{fan_{avg}}$, where $fan_{avg} = (fan_{in} + fan_{out}) / 2$
    * or a uniform dist btw $+r$ and $-r$, with $r = \sqrt{\frac{3}{fan_{avg}}}$ <br>
* activation ftn 에 따라 권장되는 weight init 방법이 조금씩 다른데 다음 테이블에 정리된다.
    | Init | Activation ftns | $\sigma^2$ |
    |---|---|---|
    | Glorot | None, tanh, logistic, softmax | $1/fan_{avg}$ |
    | He | ReLU and variants | $2/fan_{in}$ |
    | LeCun | SELU | $1/fan{in}$ |
* Keras 에서는 디폴트로 unifrom dist 의 Glorot init 을 사용한다.

In [2]:
from tensorflow import keras
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

* 만약에 $fan_{in}$ 대신 $fan_{avg}$ 를 쓰는 uniform dist He init 을 사용하고싶다면 **VarianceScaling** 을 사용한다.

In [3]:
he_avg_init = keras.initializers.VarianceScaling(scale=2.0, mode='fan_avg', distribution='uniform') # r = sqrt(3 * scale / fan_avg)
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

## 2. Nonsaturating Activation Functions
* Sigmoid activation ftn (Mother Nature in biological neurons) --> ReLU --> dyling RELUs (some nuerons die) --> variant ReLU ftn (leaky ReLU) --> ELU --> scaledELU (SELU)

* **Sigmoid** (Logistic)
    * $z = \frac{1}{1+exp(-z)}$
    * 이유: Mother Nature had chose to use roughly sigmoid activation ftns in biological neurons, they must be an excellent choice.
    * 문제점: easily saturated for positive values <br>
    <img src="images/logistic.png" width="350"/>

* **ReLU**
    * 이유: It doesn't saturate for positive values & fast to compute
    * 문제점: dying ReLUs: ReLU output 이 negative 일 때 gradient 가 0 이 되기 때문에 더이상 update 되지 않게 되고 결국 계속 0을 출력하는, "die" 즉 죽은 뉴런이 된다.
    
* **LeakyReLU**
    * $LeakyReLU_{\alpha}(z) = max(\alpha z, z)$
    * $\alpha$ is the slope for $z<0$, typical value is 0.01.
    * this small slope ensures that leaky ReLUs never die (they can go into a long coma)
    * huge leak ($\alpha=0.2$) seemed to result in better performance thatn a small leak ($alpha=0.01$) <br>
    <img src="images/LeakyReLU.png" width="350"/>
    
* **randomized leaky ReLU (RReLU)**
    * $\alpha$ is picked randomly in a given range during training and is fixed to an average val during testing.
    * regulizer 로써의 역할도 함.
    
* **parametric leaky ReLU (PReLU)**
    * $\alpha$ is authorized to be learned during training (instead of being a hyperparameter)
    * PReLU was reported to strongly outperform ReLU on large image datasets, but on smaller datasets it runs the risk of overfitting the training set.
    
* **exponential linear unit (ELU)**
    * $ELU_{\alpha} (z) = \left\{\begin{matrix}
\alpha ( exp(z) - 1)) & if \: \: z<0\\ 
z & if \:\: z\geq  0 
\end{matrix}\right. $
    * it outperformed all the ReLU variants 
    * slower to compute <br>
    <img src="images/ELU.png" width="350"/>

* **Scaled ELU (SELU)**
    * network will *self-normalize*: the output of each layer will tend to preserve a mean of 0 and stdev 1 during training.
    * RNN 이나 skip connection 과 같은 non-sequential 구조에 대해서는 SELU 의 self-normalization 을 보장할 수 없다. (그러니까 다른 activation ftn 에 비해 outperform 을 보장하지 않는다는 얘기)

* Summary
    * SELU > ELU > leaky ReLU > ReLU > tanh > logistic
    * 특정 네트워크 구조에서는 ELU > SELU (SELU 는 z = 0 에서 smooth 하지 않음)
    * run time latency 가 걱정인 사람이라면 leaky ReLU 를 선호. hyperparm 튜닝을 굳이 원치 않는다면 Keras 의 디폴트 값인 $\alpha = 0.3$ 을 그대로 사용.
    * 작업할 시간이 충분하다면 cross-validation 을 통해 여러 종류의 activation ftn 을 평가해본다.
    * overfitting 이 된다면 RReLU 고려, 학습데이터가 큰 경우 PReLU 고려
    * 스피드가 가장 중요한 요소라면 ReLU

## 3. Batch Normalization
* He init + ELU (or any variant of ReLU) 조합으로도 학습되는 동안 vanishing / exploding gradient 문제 해결을 보장할 수는 없다.
* **activation ftn 전, 또는 후**에 zero-center + normalization (현재 miniBatch 기준)
* equation
    * $\mu_B = \frac{1}{m_B} \sum_{i=1}^{m_B}{x^{(i)}}$
    * $\sigma_B ^2 = \frac{1}{m_B} \sum_{i=1}^{m_B}{(x^{(i)}- \mu_B)^2}$
    * $\hat{x}^{(i)} = \frac{x^{(i)} - \mu_B}{\sqrt{{\sigma_B}^2 + \epsilon}}$
    * $z^{(i)} = \gamma \bigotimes \hat{x}^{(i)} + \beta$ <br>
    --> rescales $\gamma$ and offsets!
* test 시에는?
    * final statistics (exponential moving average of mean and stdev) 을 사용
    * $\gamma$ and $\beta$ are trainable
    * final mean and stdev vec are used only after training.
* 특히 이미지 분류 모델에서 빠른 학습 속도와 큰 최종 성능 향상. regularization 역할도 함. 그러나 모델에 복잡도를 더함.

## 4. Gradient Clipping
* 주어진 threshold 이상의 gradient 는 clipping 해서 exploding grad 문제를 완화하는 데에 유명한 방법 중 하나다.
* Batch Norm 적용이 조금 까다로운 RNN 구조에서 자주 사용됨.
* 두 가지 방식: 
1. clipvalue (1.0 으로 셋팅)
    * org grad vector: [0.9, 100.0] 
    * after clipping: [0.9, 1.0] <br>
    --> grad vec 의 방향이 바뀜
2. clipnorm (1.0 으로 셋팅)
    * org grad vector: [0.9, 100.0]
    * after clipping: [0.00899964, 0.9999595] (0.9 / sqrt(10000.81), 100.0 / sqrt(10000.81))
    --> grad vec 의 방향 정보 유지, 하지만 첫 번째 요소가 거의 사라짐
* val set 에서 어느 방식이 더 좋은지 시도
* Keras 에서 optimizer 생성 시 *clipvalue* 또는 *clipnorm* 을 설정함

```python
# in keras
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)
# gradient vector 의 모든 값이 -1.0 에서 1.0 사이의 값이 됨.
```

# ISSUE 2. Not Enough Labeled Train Data: Reusing Pretrained Layers
## 1. Transfer Learning
* 보통 큰 데이터를 scratch 부터 시작하는건 별로 좋은 생각이 아니다. 비슷한 태스크의 뉴럴넷 모델을 찾아 거기서부터 학습하는게 다음 두 가지 측면에서 더 좋다.
    * 빠른 학습 속도
    * 적은 학습 데이터로도 학습이 가능 <br>
<img src="images/transfer_learning.png" width="350"/>
* 그림과 같이 낮은 층을 주로 재사용한다. 왜냐하면 윗층의 아웃풋은 주로 특정 태스크에 필요한 high-level feat 기 때문에 재사용하는게 별로 도움이 되지 않기 때문이다.
* 매우 비슷한 태스크라면, 모든 hidden layer 들을 재사용하고 출력층만 교체해도 된다.
* 보통 순서는 다음과 같다 
    1. freezing all the reused layers
    2. unfreezing one or two of the top hidden layers that have been reused<br>
        (만약 학습 데이터가 많다면 더 많은 층을 unfreeze 시켜도 된다.)<br>
        (unfreeze 시 learning rate 을 줄인다.)<br>
* Keras implementation
    * model A (existing): > 90 % accuracy on 8 classes classification tast
    * model B: 97.2 % accuracy on binary classifcation test (model A 와 같은 구조지만 더 쉬운 태스크기 때문에 더 높은 정확도를 얻음.)
    * model A 를 transfer learning 하면 97.2 % accuracy 보다 더 좋은 성능을 낼 수 있을까?
    
```python
# in keras
# model_A 와 model_B_on_A 는 마지막 층 제외 weights 를 공유한다. 
model_A = keras.models.load_model("mymodel_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# 하지만 model_B 를 학습할 때 model_A 도 함께 업데이트되는 문제 발생한다.
# 따라서 다음과 같이 *clone_model()* 을 사용해서 model_A 의 weight 를 복사한다.
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights()) # clone_model() 이 weight 까지 복사하는 것은 아니기 때문에 set_weight 해주어야한다.

# final layer 로 인한 큰 에러로 인해 재사용되는 층들이 어그러지는 것을 방지하기 위해 freeze 시킨다.
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd", metrics=["accuracy"]) # you must always compile your model after you freeze or unfreeze layers.

# 몇 에폭 동안 학습한 후에, 재사용 층을 unfreeze 한 후 마저 학습한다. (재사용 층은 fine tuned 될 것이다.)
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, validation_data=(X_valid_B, y_valid_B))
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True
optimizer = keras.optimizers.SGD(lr=1e-4) # the defualt lr is 1e-2
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, validation_data=(X_valid_B, y_valid_B))
model_B_on_A.evaluate(X_test_B, y_test_B)
```

* 최종 결과: [0.068879..., 0.9925], 즉 accuracy 99.25 % (2 % 오름!)
* transfer learning 은 deep CNN 에서 제일 잘 working 함 (small net 이나 dense net 보다는)

## 2. Unsupervised Pretraining
* 언제 사용?: 1. 부족한 labeled data, 2. 충분한 unlabeled data, 3. no model trained on a similar task (transfer learning 어려움.)
* 과정
    * Use unlabeled training data to train unsupervised model (autoencoder, GAN (RBMs in old days))
    * Reuse the lower layers + Add the output layer for your task on top
    * fine-tune the final ent using supervised learning w/ the labeled training data. <br>
    <img src="images/unsupervised_pretraining.png" width="450"/>

## 3. Pretraining on an Auxiliary Task
* labeled data 가 부족할 때 사용할 수 있는 또 다른 방법.

# ISSUE 3. 느린 학습 속도
## 1. Momentum Optimization
## 2. Nesterov Accelerated Gradient
## 3. AdaGrad
## 4. RMSProp
## 5. Adam and Nadam Optimization
## 6. Learning Rate Scheduling

# ISSUE 4. Overfitting
## 1. $l_1$ and $l_2$ Regularization
## 2. Dropout
## 3. Monte Carlo (MC) Dropout
## 4. Max-Norm Regularization

# Summary and Practical Guidelines

#### To do 
* Issue 3, 4